In [ ]:
# Fit a linear regression model (name the model "linreg").
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()


feature_cols = ['al']
X = glass[feature_cols]
y = glass.ri
linreg.fit(X, y)

In [ ]:
print(linreg.intercept_)
print(linreg.coef_)

# Compute prediction for al=2 using the predict method.
linreg.predict(2)

In [ ]:
#Mapping if want to use logistic regression. Result is either 1 or 0. So pred values can be seen as

# Types 1, 2, 3 are window glass.
# Types 5, 6, 7 are household glass.
glass['household'] = glass.glass_type.map({1:0, 2:0, 3:0, 5:1, 6:1, 7:1})
glass.head()

# Transform household_pred to 1 or 0.
glass['household_pred_class'] = np.where(glass.household_pred >= 0.5, 1, 0)
glass.head()

In [ ]:
logreg.predict_proba(X)[0:10] #probability of 0 vs 1

In [ ]:
#Confusion Matrix
metrics.confusion_matrix(y_true=y_test, y_pred=logit_pred_proba > .3)

#ROC/AUC

Decision Tree

In [1]:
# Define a function that calculates the RMSE for a given split of miles.
def mileage_split(miles):
    lower_mileage_price = train[train.miles < miles].price.mean()
    higher_mileage_price = train[train.miles >= miles].price.mean()
    train['prediction'] = np.where(train.miles < miles, lower_mileage_price, higher_mileage_price)
    return np.sqrt(metrics.mean_squared_error(train.price, train.prediction))

# Calculate RMSE for tree that splits on miles < 50,000.
print('RMSE:', mileage_split(50000))
train

NameError: name 'train' is not defined

In [ ]:
# Check all possible mileage splits.
mileage_range = list(range(train.miles.min(), train.miles.max(), 1000)) #start with min, increment by 1000, end with max
RMSE = [mileage_split(miles) for miles in mileage_range]


# Plot mileage cutpoint (x-axis) versus RMSE (y-axis).
plt.plot(mileage_range, RMSE);
plt.xlabel('Mileage cutpoint');
plt.ylabel('RMSE (lower is better)');

#Find the one with lowest RMSE

## Building Regression Tree
Split based on reducing MSE

In [ ]:
# Define X and y.
feature_cols = ['year', 'miles', 'doors', 'vtype']

X = train[feature_cols]
y = train.price


# Instantiate a DecisionTreeRegressor (with random_state=1).
from sklearn.tree import DecisionTreeRegressor

treereg = DecisionTreeRegressor(random_state=1,max_depth=40)  # change the max depth to tune
treereg

# Use leave-one-out cross-validation (LOOCV) to estimate the RMSE for this model.#CV for decision tree
from sklearn.model_selection import cross_val_score
scores = cross_val_score(treereg, X, y, cv=14, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

In [ ]:
# Tuning of Max DEpth. You try different depth and see the MSE

# List of values to try:
max_depth_range = list(range(1, 8))

# List to store the average RMSE for each value of max_depth:
RMSE_scores = []

# Use LOOCV with each value of max_depth.
for depth in max_depth_range:
    treereg = DecisionTreeRegressor(max_depth=depth, random_state=1)
    MSE_scores = cross_val_score(treereg, X, y, cv=14, scoring='neg_mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

    
#Then plot graph to see RMSE agst Max Depth
# Plot max_depth (x-axis) versus RMSE (y-axis). Find the depth with lowest MSE
plt.plot(max_depth_range, RMSE_scores);
plt.xlabel('max_depth');
plt.ylabel('RMSE (lower is better)');



# Sort and find the max depth to use and corresponding RMSE
sorted(zip(RMSE_scores, max_depth_range))[0]



#Then fit the tree with the optimal depth
# max_depth=3 was best, so fit a tree using that parameter.
treereg = DecisionTreeRegressor(max_depth=3, random_state=1)
treereg.fit(X, y)

In [ ]:
#Once tree model is built
# "Gini importance" of each feature: the (normalized) total reduction of error brought by that feature.
pd.DataFrame({'feature':feature_cols, 'importance':treereg.feature_importances_})

In [ ]:
#Lastly create tree diagram 
# Create a Graphviz file.
from sklearn.tree import export_graphviz
export_graphviz(treereg, out_file='./assets/tree_vehicles.dot', feature_names=feature_cols)

# At the command line, run this to convert to PNG:
#   dot -Tpng tree_vehicles.dot -o tree_vehicles.png

# Or, you can drag the image below to your desktop or Powerpoint!

In [ ]:
#BUild model already, test prediction on test set to calculate performance using MSE

X_test = test[feature_cols] #df
y_test = test.price
y_pred = treereg.predict(X_test)

y_pred

# Calculate RMSE
np.sqrt(metrics.mean_squared_error(y_test, y_pred))

## Building Classification Tree
Split based on Minimising GINI index

In [ ]:
# Define X and y.
feature_cols = ['Pclass', 'Sex', 'Age', 'Embarked_Q', 'Embarked_S']

X = titanic[feature_cols]
y = titanic.Survived


# Fit a classification tree with max_depth=3 on all data.
from sklearn.tree import DecisionTreeClassifier

treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

In [ ]:
# Create a Graphviz file.
export_graphviz(treeclf, out_file='./assets/tree_titanic.dot', feature_names=feature_cols)

# At the command line, run this to convert to PNG:
#   dot -Tpng tree_titanic.dot -o tree_titanic.png

# Or, just drag this image to your desktop or Powerpoint.

In [ ]:
# Compute the feature importances (the Gini index at each node).

pd.DataFrame({'feature':feature_cols, 'importance':treeclf.feature_importances_})

## Bagging of decision tree

Reduces variance which increases accuracy. Usually for decision tree. Can be for other models

Typical decision tree very high variance (ie diff model to model becos based on how it is being split.)

If just want to use bagging for random forest, can just use random forest regressor. NO need bagging regressor then choose decision tree regressor

In [ ]:
# Define the training and testing sets.
X_train = train.iloc[:, 1:]
y_train = train.iloc[:, 0]
X_test = test.iloc[:, 1:]
y_test = test.iloc[:, 0]

In [ ]:
#Instruct BaggingRegressor to use DecisionTreeRegressor as the "base estimator."

#Bagging can be applied to other methods so need to choose decision tree regressor

from sklearn.ensemble import BaggingRegressor
bagreg = BaggingRegressor(DecisionTreeRegressor(), n_estimators=500, bootstrap=True, oob_score=True, random_state=1)

In [ ]:
# Fit and predict.
bagreg.fit(X_train, y_train)
y_pred = bagreg.predict(X_test)
y_pred

# Calculate RMSE.
np.sqrt(metrics.mean_squared_error(y_test, y_pred))

#Still can generate feature importance. But unable to view visuallyGladiwn89


## Random Forest

In [ ]:
#Classfication: M (where M<P) is the sqrt of P (full number of features)
#Regression, m is typically chosen to be somewhere between p/3 and p.

from sklearn.ensemble import RandomForestRegressor
# max_features=5 is best and n_estimators=150 is sufficiently large.# n_estimators is the number of trees you want to build
# More the merrier but slowing to train
rfreg = RandomForestRegressor(n_estimators=150, max_features=5, oob_score=True, random_state=1)
rfreg.fit(X, y)

# Compute feature importances.
pd.DataFrame({'feature':feature_cols, 'importance':rfreg.feature_importances_}).sort_values(by='importance')

# Compute the out-of-bag R-squared score. HIgher the better. Means run the unseen data on the ensemble to check accuracy
print((rfreg.oob_score_))

# Find the average RMSE. #Auto take care of indivudal Y pred and Ytest
scores = cross_val_score(rfreg, X, y, cv=10, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))



### Feature selection - split into train and test first before doing feature selection

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state = 89)


# Fit the model on only the train data
rfreg = RandomForestRegressor(n_estimators=150, max_features=5, oob_score=True, random_state=1)
rfreg.fit(X_train, y_train)


# Set a threshold for which features to include. # include anything with importance > mean/median
from sklearn.feature_selection import SelectFromModel

print(SelectFromModel(rfreg, threshold='mean', prefit=True).transform(X_train).shape)
print(SelectFromModel(rfreg, threshold='median', prefit=True).transform(X_train).shape)

In [2]:
# Create a new feature matrix that only includes important features on TEST X set

X_important =  SelectFromModel(rfreg, threshold='mean', prefit=True).transform(X_test)

#Run prediction using new X test which is X important. Then check avg RMSE
scores = cross_val_score(rfreg, X_important, y_test, cv=10, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

NameError: name 'SelectFromModel' is not defined

### Tune No of Tree

In [3]:
# List of values to try for n_estimators:
estimator_range = list(range(10, 310, 10))

# List to store the average RMSE for each value of n_estimators:
RMSE_scores = []

# Use five-fold cross-validation with each value of n_estimators (Warning: Slow!).
for estimator in estimator_range:
    rfreg = RandomForestRegressor(n_estimators=estimator, random_state=1)
    MSE_scores = cross_val_score(rfreg, X, y, cv=5, scoring='neg_mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))
    

# Then plot RMSE (y-axis) versus n_estimators (x-axis) and find optimal N_estimators

plt.plot(estimator_range, RMSE_scores);

plt.xlabel('n_estimators');
plt.ylabel('RMSE (lower is better)');

#use the Zipped and sort to find the lowest MSE and corresponding N_estimators
# Show the best RMSE and the corresponding max n_estimators
sorted(zip(RMSE_scores, estimator_range))[0]

NameError: name 'RandomForestRegressor' is not defined

### Tune max features

In [ ]:
# List of values to try for max_features:
feature_range = list(range(1, len(feature_cols)+1))

# List to store the average RMSE for each value of max_features:
RMSE_scores = []

# Use 10-fold cross-validation with each value of max_features (Warning: Super slow!).
for feature in feature_range:
    rfreg = RandomForestRegressor(n_estimators=150, max_features=feature, random_state=1)
    MSE_scores = cross_val_score(rfreg, X, y, cv=10, scoring='neg_mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

In [ ]:
# Plot max_features (x-axis) versus RMSE (y-axis).

plt.plot(feature_range, RMSE_scores);

plt.xlabel('max_features');
plt.ylabel('RMSE (lower is better)');

# Show the best RMSE and the corresponding max_features.
sorted(zip(RMSE_scores, feature_range))[0]

# Boosting

In [ ]:
# random search logistic regression model on the sonar dataset
from scipy.stats import loguniform
from pandas import read_csv
import numpy as np
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [ ]:
#Gradient Boosting REgessor
reggbm = GradientBoostingRegressor(max_depth=3,  n_estimators=50, random_state=0)
reggbm.fit(X,Y.values.ravel())
Y_pred=reggbm.predict(X)
print(r2_score(Y,Y_pred))

# Calculate RMSE.
print(np.sqrt(metrics.mean_squared_error(Y,Y_pred)))

In [ ]:
#Ada Boost
rega = AdaBoostRegressor( n_estimators=50, random_state=0)
rega.fit(X,Y.values.ravel())
Y_pred=rega.predict(X)
print(r2_score(Y,Y_pred))

# Calculate RMSE.
print(np.sqrt(metrics.mean_squared_error(Y,Y_pred)))

# Clustering

### K Means

In [4]:
%matplotlib inline 

import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from sklearn import cluster, datasets, preprocessing, metrics
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

In [ ]:
#Pairplots
cols = df.columns[:-1]
sns.pairplot(df[cols])

In [ ]:
#Min max scalling. Features might have different ranges, hence normalise
X_scaled = preprocessing.MinMaxScaler().fit_transform(df[cols])


#Another way to scale using standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
#Set K, build model
k = 2
kmeans = cluster.KMeans(n_clusters=k)
kmeans.fit(X_scaled)


#Find the labels, centroids, inertia (distance between points in same cluster. Smaller the better)
labels = kmeans.labels_
centroids = kmeans.cluster_centers_   #will have an array for each of the feature
inertia = kmeans.inertia_

#Find Silhouette (distance between clusters. BIgger the better)
metrics.silhouette_score(X_scaled, labels, metric='euclidean')

In [ ]:
#Prediction is by allocating the labels after finding labels
df['label'] = labels
beer.sort_values('cluster')

In [ ]:
#Plot diagram
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 14


plt.scatter(beer.calories, beer.alcohol, c=colors[beer.cluster], s=50);

# Cluster centers, marked by "+"
plt.scatter(centers.calories, centers.alcohol, linewidths=3, marker='+', s=300, c='black');

# Add labels
plt.xlabel('calories');
plt.ylabel('alcohol');